# First Spark Streaming Example
_____

# Twitter Example
Set up the credentials for a twitter app at https://apps.twitter.com/
    
install python-twitter, a python library to connect your Python to the twitter dev account.

Begin by running the TweetRead.py file. Make sure to add your own IP Adress and your credential keys.

In [1]:
import findspark

In [2]:

findspark.init('/Users/kevinblum/Apache-Spark/spark-3.1.2-bin-hadoop3.2')

In [3]:

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
from pyspark.ml import PipelineModel
from pyspark.ml.feature import CountVectorizer

In [4]:
spark = spark = SparkSession \
    .builder \
    .appName("Twitter Streaming App") \
    .getOrCreate()

tweet_df = spark \
    .readStream \
    .format("socket") \
    .option("host", "127.0.0.1") \
    .option("port", 5556) \
    .load()

tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")

In [5]:
tweet_df_string = tweet_df_string.withColumnRenamed("value" , "tweet")

In [6]:
writeTweet = tweet_df_string.writeStream.format("csv").\
    option("format" , "append").\
    option("path", "/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets").\
    option("checkpointLocation" , "checkpoints").\
    queryName("tweetquery"). \
    start()


In [7]:
import time
time.sleep(60) 
writeTweet.stop()

In [8]:
import pandas as pd

In [9]:
import os
import glob
import pandas as pd
os.chdir("/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets")

In [10]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]


In [11]:
dataframe = pd.DataFrame()
for i in all_filenames:
    if (os.stat(f"/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets/{i}").st_size != 0):
        try:
            d = pd.read_csv(f"/Users/kevinblum/BigDataProj/SparkTwitterStream/tweets/{i}" , engine="python")
            dt = d.T
            dt = dt.reset_index()
            dataframe = dataframe.append(dt)
        except:
            pass




In [12]:
dataframe = dataframe.rename(columns={"index" : "Tweet"})
dataframe = dataframe.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
dataframe = dataframe.reset_index(drop=True)

In [13]:

dataframe = dataframe[dataframe['Tweet'] != ""]
dataframe = dataframe[dataframe['Tweet'] != "."]
dataframe = dataframe[dataframe['Tweet'] != None]

dataframe = dataframe.reset_index(drop=True)


In [14]:
range1 = range(100, 100 + dataframe['Tweet'].size  )
list1 = list(range1)


In [15]:
dfg = pd.DataFrame(list1 , columns = ['tweet_id'])

In [16]:
dfg = dfg['tweet_id'].astype('str')

In [17]:
list_polarity =[]
for b in range(dataframe['Tweet'].size):
    list_polarity.append(0)
    

In [18]:
dfp = pd.DataFrame(list_polarity , columns = ['polarity'])

In [19]:
dataframe = dataframe.join(dfg)

In [20]:
dataframe = dataframe.join(dfp)

In [21]:
dataframe

,Tweet,tweet_id,polarity
0,Proving a point.RT @achyuta_samanta: Attended ...,100,0
1,A victory for hope and change.,101,0
2,I also find it amusing that Sen. Manchin indic...,102,0
3,No Yes Maybe,103,0
4,"If you hear nothing else I say tonight, hear t...",104,0
...,...,...,...
79,5 Winners will be chosen in the morning,179,0
80,https://t.co/LI75LKV2Ty https://t.co/8G0ApWKqJ...,180,0
81,"The President is not playing,",181,0
82,#RobloxDevRT @joncoopertweets: Retweet if you ...,182,0


In [22]:
sparkDF = spark.createDataFrame(dataframe)

In [23]:
sparkDF.show(5)

+--------------------+--------+--------+
|               Tweet|tweet_id|polarity|
+--------------------+--------+--------+
|Proving a point.R...|     100|       0|
|A victory for hop...|     101|       0|
|I also find it am...|     102|       0|
|No       Yes     ...|     103|       0|
|If you hear nothi...|     104|       0|
+--------------------+--------+--------+
only showing top 5 rows



In [24]:
import nltk

In [25]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [26]:
from nltk.corpus.reader.wordnet import *
from pyspark.sql.functions import udf
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
from pyspark.sql.types import StringType,DoubleType,IntegerType
import pyspark.sql.functions as F
wn = nltk.WordNetLemmatizer()
worddict = set(nltk.corpus.words.words())


def preprocessing(text):
    wordset_n = set(wn.lemmatize(w, NOUN) for w in word_tokenize(text.lower().strip()))
    wordset_v = set(wn.lemmatize(w, VERB) for w in wordset_n)
    wordset = set(wn.lemmatize(w, ADJ) for w in wordset_v)
    wordset = wordset & worddict
    return ' '.join(list(wordset))


brand_udf=udf(preprocessing,StringType())
sparkDF=sparkDF.withColumn('text',brand_udf(sparkDF['Tweet']))


In [27]:
sparkDF.show(2)

+--------------------+--------+--------+--------------------+
|               Tweet|tweet_id|polarity|                text|
+--------------------+--------+--------+--------------------+
|Proving a point.R...|     100|       0|meet a prove asso...|
|A victory for hop...|     101|       0|a for victory cha...|
+--------------------+--------+--------+--------------------+
only showing top 2 rows



In [31]:
from pyspark.ml import PipelineModel

model_1 = PipelineModel.load("/Users/kevinblum/BigDataProj/SparkTwitterStream/models/lr")


In [32]:
predictions = model_1.transform(sparkDF)

In [33]:
predictions1=predictions.toPandas()

In [35]:
predictions1.head()

,Tweet,tweet_id,polarity,text,words,rawFeatures,features,rawPrediction,probability,prediction
0,Proving a point.RT @achyuta_samanta: Attended ...,100,0,meet a prove association attend president annu...,"[meet, a, prove, association, attend, presiden...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.725029883894388, -6.311348320862588, -0.770...","[0.12524714839733533, 7.420534871122743e-07, 0...",4.0
1,A victory for hope and change.,101,0,a for victory change hope and,"[a, for, victory, change, hope, and]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[6.613612295761145, -6.303998354041012, -2.662...","[0.11472971318733188, 2.815975573540839e-07, 1...",4.0
2,I also find it amusing that Sen. Manchin indic...,102,0,deficit appoint i that for un to group senator...,"[deficit, appoint, i, that, for, un, to, group...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[2.2126436601947974, -6.349880681323569, -5.17...","[1.4419253025753738e-06, 2.756041102083589e-10...",4.0
3,No Yes Maybe,103,0,no yes maybe,"[no, yes, maybe]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.601525388731286, -6.3002229502612135, -2.22...","[0.5943576635546819, 5.452494979576245e-07, 3....",0.0
4,"If you hear nothing else I say tonight, hear t...",104,0,name i that anyone for last nothing forget joe...,"[name, i, that, anyone, for, last, nothing, fo...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[7.918591595913286, -6.328716729734098, -4.839...","[0.15978347674940224, 1.0375391161748201e-07, ...",4.0


In [ ]:
from pyspark.ml.feature import  Tokenizer

In [ ]:
tokenizer = Tokenizer(inputCol="text", outputCol="words")

In [ ]:
sparkDF=tokenizer.transform(sparkDF)

In [ ]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")

In [ ]:
model1=count.fit(sparkDF)

In [ ]:
sparkDF=model1.transform(sparkDF)

In [ ]:
from pyspark.ml.feature import  IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [ ]:
idfModel = idf.fit(sparkDF)
sparkDF = idfModel.transform(sparkDF)

In [ ]:

sparkDF=sparkDF.repartition(10)

In [ ]:
sparkDF.printSchema()

In [ ]:
sparkDF = sparkDF.withColumn('polarity', sparkDF['polarity'].cast(IntegerType()))

In [ ]:
sparkDF.show(10)

In [ ]:
from pyspark.ml.classification import RandomForestClassificationModel
rf_model = RandomForestClassificationModel.load("/Users/kevinblum/BigDataProj/SparkTwitterStream/models/rf_classifier")

In [ ]:
sparkDF

In [ ]:
rf_predictions=rf_model.transform(sparkDF.select(['text','words','rawFeatures','features','polarity']))

In [ ]:
rf_predictions.select(['prediction']).show(10)

In [ ]:
import pyspark.sql.functions as F

print(F.length("polarity"))

In [ ]:
sparkDF

In [ ]:
rd_predict = rf_predictions.withColumn("prediction" , rf_predictions["prediction"].cast(StringType()))